<a href="https://colab.research.google.com/github/Itskindastrange/movie-review-analysis/blob/main/movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import imdb
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pandas as pd

In [3]:
VOCAB_SIZE = 88584
MAX_LEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

In [4]:
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=MAX_LEN)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=MAX_LEN)

#Model

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [6]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['acc']
    )

In [7]:
history = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=BATCH_SIZE,
    validation_split=0.2
)

Epoch 1/10
313/313 [==============================] - 49s 150ms/step - loss: 0.5157 - acc: 0.7304 - val_loss: 0.3348 - val_acc: 0.8592
Epoch 2/10
313/313 [==============================] - 48s 154ms/step - loss: 0.2831 - acc: 0.8880 - val_loss: 0.4395 - val_acc: 0.8490
Epoch 3/10
313/313 [==============================] - 46s 147ms/step - loss: 0.2066 - acc: 0.9227 - val_loss: 0.3015 - val_acc: 0.8810
Epoch 4/10
313/313 [==============================] - 45s 143ms/step - loss: 0.1648 - acc: 0.9418 - val_loss: 0.3092 - val_acc: 0.8858
Epoch 5/10
313/313 [==============================] - 45s 144ms/step - loss: 0.1345 - acc: 0.9542 - val_loss: 0.3591 - val_acc: 0.8612
Epoch 6/10
313/313 [==============================] - 48s 155ms/step - loss: 0.1087 - acc: 0.9628 - val_loss: 0.4769 - val_acc: 0.8380
Epoch 7/10
313/313 [==============================] - 45s 145ms/step - loss: 0.0902 - acc: 0.9710 - val_loss: 0.5468 - val_acc: 0.8170
Epoch 8/10
313/313 [==============================] - 4

In [8]:
model.evaluate(test_data, test_labels)

782/782 [==============================] - 20s 26ms/step - loss: 0.5423 - acc: 0.8602


[0.5423004031181335, 0.8601599931716919]

#Making Pred

In [9]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return tf.keras.preprocessing.sequence.pad_sequences([tokens], maxlen=MAX_LEN)[0]

1641221/1641221 [==============================] - 0s 0us/step


In [10]:
text = "that movie was just amazing, so amazing"
encoded = encode_text(text)

In [11]:
print('Encoded text:', encoded)

Encoded text: [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0 

In [14]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "

  return text[:-1]

In [15]:
print('Decoded review:', decode_integers(encoded))

Decoded review: that movie was just amazing so amazing


In [19]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)

  if result[0] > 0.5:
    print('Positive review')
  else:
    print('Negative review')

In [22]:
positive_review = "That movie was so good.I really loved it, I'll recommend it to my friends"
predict(positive_review)

1/1 [==============================] - 0s 30ms/step
Positive review


In [21]:
negative_review = "That movie was so bad"
predict(negative_review)

1/1 [==============================] - 0s 98ms/step
Negative review
